In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch==2.1.0 torchvision==0.16.0 --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [ ]:
!pip install -q pdfplumber  #for extract text from pdf

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines import pipeline

data_dir = '/content/drive/My Drive/Resume/data/data/'

# List categories (folder names)
categories = os.listdir(data_dir)
print("Found categories:", categories)

Found categories: ['FINANCE', 'PUBLIC-RELATIONS', 'SALES', 'HR', 'TEACHER', 'INFORMATION-TECHNOLOGY', 'FITNESS', 'HEALTHCARE', 'AVIATION', 'BUSINESS-DEVELOPMENT', 'DIGITAL-MEDIA', 'BANKING', 'CONSULTANT', 'CHEF', 'BPO', 'ENGINEERING', 'CONSTRUCTION', 'DESIGNER', 'AGRICULTURE', 'ACCOUNTANT', 'ARTS', 'AUTOMOBILE', 'APPAREL', 'ADVOCATE']


In [ ]:
from transformers.models.auto.tokenization_auto import AutoTokenizer
from transformers.models.auto.modeling_auto import AutoModelForCausalLM
from transformers.pipelines import pipeline
import torch

model_name = "tiiuae/falcon-7b-instruct"

try:
    print(f"Loading model: {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16 # Use bfloat16 for reduced memory usage
    )

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    print("Model loaded successfully!")

except Exception as e:
    print(f"Error loading model or creating pipeline: {e}")

Loading model: tiiuae/falcon-7b-instruct...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model loaded successfully!


In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text.strip()


In [ ]:
!pip install pdfplumber
import pdfplumber


In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text.strip()


In [ ]:
from google.colab import files
import os

uploaded = files.upload()

for filename in uploaded.keys():
    resume_path = filename
    resume_text = extract_text_from_pdf(resume_path)

    if resume_text:
        prompt = f"""
Rate the following resume out of 10 and explain your reasoning clearly.

Resume:
{resume_text}
"""

        print(f"\n📄 Evaluating: {filename}\n")
        result = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)[0]['generated_text']
        print("🤖 Falcon's Response:\n", result)
    else:
        print(f"⚠️ Could not extract text from: {filename}")


Saving 10554236.pdf to 10554236 (1).pdf


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



📄 Evaluating: 10554236 (1).pdf



RuntimeError: Numpy is not available

In [ ]:
import re
import pandas as pd

# Initialize a list to store results
results = []

# Run evaluation on a single uploaded resume
for filename in uploaded.keys():
    resume_path = filename
    resume_text = extract_text_from_pdf(resume_path)

    if resume_text:
        prompt = f"""
Rate the following resume out of 10 and explain your reasoning.

Resume:
{resume_text}
"""

        print(f"\n📄 Evaluating: {filename}\n")
        response = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)[0]['generated_text']
        print("🤖 Falcon's Response:\n", response)

        # Extract the score (e.g., "8 out of 10" or "I would rate it 9/10")
        score_match = re.search(r'(\b\d{1,2}\b)\s*/?\s*10', response)
        score = int(score_match.group(1)) if score_match else None

        results.append({
            'filename': filename,
            'score': score,
            'response': response
        })

    else:
        print(f"⚠️ Could not extract text from: {filename}")

# Convert to DataFrame
df_scores = pd.DataFrame(results)
df_scores
